# Circuit - Python SDK

This notebook is intended to demonstrate the features of this sdk.


Getting Started to use this notebook:
- Open a terminal and cd into this example directory
- Run 'uv sync'
- Activate the newly created .venv for your jupyter notebook
  - This step may vary, but if you are using VS Code or equivalent, open your command pallete and select the Python Intrepreter selection option, then enter the path of the generated .venv/bin/python that was created during the uv sync command. This should allow you to select this intrepreter for your jupyter notebook.
  - If you are not using VS Code or a fork of it (cursor for example), simply google how to select the python intrepreter if you have having issues doing so
- Tip: You should have hover tooltips for this sdk, so if you are seeing 'any', double check your ide setup

In [1]:
import json

# This will be passed into your execution and stop functions automatically
# Feel free to use this in your development process for your own testing needs
# pydantic models are provided to ensure proper usage of the SDK without needing to reference docs separately
from agent_sdk import AgentContext

# Pydantic models - helpful for ensuring data integrity during development, you are not required to use this
# in your agent's code, it is simply provided to help guide you when creating sample data for development
from agent_sdk.agent import CurrentPosition

##### Setting up a mock request (AgentContext) for your agent

In [7]:
# Right now, the easiest way to get a session id and wallet address for development is to use the cli using the following options
# Option 1: You have not created a session yet in the cli
#     - Simply put an empty return at the very top of your execute function, and run 'circuit run -m local -x execute', and note down the logged session Id and the address you used
#     - Commenting out your agent code is completely optional for that step, it is just recommended so you dont have to run actual logic to simply generate the session Id
#     - This will be fixed in the cli soon to simply create a development session
# Option 2: You have already created a session in the cli
#     - Open up the ~/.config/circuit/config.json file and get the sessionId and walletAddress from there

SESSION_ID = 7393  # update with your session id
WALLET_ADDRESS = "<your_wallet_address_goes_here>"

# Use this to create sample "positions" to test your agent's logic for managing existing positions
# Your agent will only be able to transact with assets that appear here in each request, this is also referred to as the agent's allocation
# You will only need to track the 'intra-execution' transaction changes, as the circuit infra will send the updated positions on each subsequent run
# The first run for any session will always return the requried asset that you configured during the init step in the cli + plus $2 of gas if the required asset is not the native asset of the network
# For example, if you configured the required asset to be USDC.e on polygon (matic), then the initial run will have USDC + $2 of POL. If you set the required asset to POL, then only that asset will be returned
# It is up to the agent developer to manage gas as transactions are made, if more than the initial $2 of gas is needed. (This will be changing in the near future to be network-specific)
SAMPLE_POSITION = CurrentPosition(
    network="ethereum:137",
    assetAddress="0x4d97dcd97ec945f40cf65f87097ace5ea0476045",
    tokenId="86192057611122246511563653509192966169513312957180910360241289053249649036697",
    avgUnitCost="0.779812797920499100",
    currentQty="41282044",
)

agent = AgentContext(
    sessionId=SESSION_ID,
    sessionWalletAddress=WALLET_ADDRESS,
    currentPositions=[SAMPLE_POSITION],
)

# Core Functions

> Note: All agent.* functions will return with a success boolean and will not throw any errors, and instead will provide any error messages in the returned function data

## Logging

The SDK includes an opinionated set of logging functions to manage communication with the end user, as well as logging to the console. Its advised to use this over the default python logging to avoid duplicating code and making sure you are sending quality messages to the users.

#### Default - Local output + message sent to the API to be viewed on the frontend

In [26]:
# no throwaway variable
agent.log("Test")

2025-10-14 16:35:50 - circuit_agent - INFO - Test


LogResponse(success=True, error=None, error_details=None)

In [4]:
# with throwaway
_ = agent.log("Test", debug=True)

2025-10-14 12:44:03 - circuit_agent - INFO - Test


In [7]:
sample_user_log = agent.log("Hello, world! - local dev output + user message")

2025-10-10 16:30:19 - circuit_agent - INFO - Hello, world! - local dev output + user message


#### Send error message to the user

In [4]:
#### Error message sent to user to be viewed on the frontend
sample_error_log = agent.log(
    message="Test Error message - this will be shown in the UI as an error during execution",
    error=True,
)

2025-10-13 15:40:00 - circuit_agent - ERROR - Test Error message - this will be shown in the UI as an error during execution


#### Local output only, only logged to console
- Passing debug=True will stop logs from being sent to the API and will be logged locally in your console only 
- This also applies if you pass the error=True property

In [7]:
sample_local_log = agent.log(
    message="Hello, world! - local dev output only", debug=True
)

2025-10-11 21:20:28 - circuit_agent - INFO - Hello, world! - local dev output only


##### Handles dicts and pydantic models - both will be dumped to strings before being sent to the backend and truncated to 250 characters

In [6]:
pydantic_model_log = agent.log(message=[SAMPLE_POSITION, SAMPLE_POSITION], debug=True)

2025-10-11 21:19:27 - circuit_agent - INFO - [
  "network='ethereum:137' assetAddress='0x4d97dcd97ec945f40cf65f87097ace5ea0476045' tokenId='86192057611122246511563653509192966169513312957180910360241289053249649036697' avgUnitCost='0.779812797920499100' currentQty='41282044'",
  "network='ethereum:137' assetAddress='0x4d97dcd97ec945f40cf65f87097ace5ea0476045' tokenId='86192057611122246511563653509192966169513312957180910360241289053249649036697' avgUnitCost='0.779812797920499100' currentQty='41282044'"
]


In [5]:
import datetime

nested_dict = {
    "dateTimeValue": datetime.datetime.now(),
    "wallet": agent.sessionWalletAddress,
    "nested": {"key": "value"},
}
sample_json_log = agent.log(nested_dict, debug=True)

2025-10-11 21:19:24 - circuit_agent - INFO - {
  "dateTimeValue": "2025-10-11 21:19:24.922240",
  "wallet": "0xf92b294f79e44f12279db546ed87557c6bd1bc02",
  "nested": {
    "key": "value"
  }
}


## Sign and Send (Custom transactions)
This is our lowest level function for sending transactions. If you need to perform an on-chain action that is not covered by our Swidge or Polymarket functions, you can build your own data and use this function

In [3]:
sign_and_send = agent.sign_and_send(
    {
        "network": "ethereum:1",
        "request": {
            "to_address": agent.sessionWalletAddress,
            "data": "0x",  # Self send
            "value": "100000000000000",  # 0.0001 ETH
        },
        "message": "Self-send demo",
    }
)

In [4]:
_ = agent.log(sign_and_send, debug=True)

2025-10-13 16:30:26 - circuit_agent - INFO - {
  "success": true,
  "internal_transaction_id": 65674,
  "tx_hash": "0xef2d63ea27a6f6cc0d85708da24e25da0c29c80b5a724a2c601905746368acc4",
  "transaction_url": "https://etherscan.io/tx/0xef2d63ea27a6f6cc0d85708da24e25da0c29c80b5a724a2c601905746368acc4",
  "error": null,
  "error_details": null
}


## Transaction History
List of asset changes for all transactions during a given session. 

> Note: The system needs to index new transactions, so there may be a slight delay between when you execute a transaction and when the resulting asset changes are returned in this method. Make sure you are taking that into consideration if dealing with assets the agent just transacted with.

In [ ]:
transaction_history = agent.transactions()
agent.log(transaction_history, debug=True)

## Current Positions

In [ ]:
current_positions = agent.get_current_positions()
agent.log(current_positions, debug=True)

# Agent/Session Memory

#### Setting/Updating Memory

In [15]:
agent.memory.get("doesnt-exist")

MemoryGetResponse(success=False, data=None, error='Key not found', error_details={'success': False, 'error': 'Key not found'})

In [5]:
memory_set = agent.memory.set(
    "thing-to-store", json.dumps({"test-key": "test-value-new"})
)
_ = agent.log(memory_set, debug=True)

if not memory_set.success:
    _ = agent.log(
        f"Memory set failed: {memory_set.error_message}", error=True, debug=True
    )

2025-10-13 16:21:21 - circuit_agent - INFO - {
  "success": true,
  "data": {
    "key": "thing-to-store"
  },
  "error": null,
  "error_details": null
}


#### List Memory Items

In [6]:
memory_list = agent.memory.list()
_ = agent.log(memory_list, debug=True)

if not memory_list.success:
    _ = agent.log(
        f"Memory list failed: {memory_list.error_message}", error=True, debug=True
    )

2025-10-13 16:21:24 - circuit_agent - INFO - {
  "success": true,
  "data": {
    "keys": [
      "thing-to-store"
    ],
    "count": 1
  },
  "error": null,
  "error_details": null
}


#### Get Memory Item

In [7]:
memory_item = agent.memory.get("thing-to-store")
_ = agent.log(memory_item, debug=True)

if not memory_item.success:
    _ = agent.log(
        f"Memory item retrieval failed: {memory_item.error_message}",
        error=True,
        debug=True,
    )

2025-10-13 16:21:28 - circuit_agent - INFO - {
  "success": true,
  "data": {
    "key": "thing-to-store",
    "value": "{\"test-key\": \"test-value-new\"}"
  },
  "error": null,
  "error_details": null
}


#### Delete Memory

In [8]:
for key in memory_list.data.keys:
    deleted_memory_item = agent.memory.delete(key)
    if deleted_memory_item.success:
        _ = agent.log(f"Memory deleted: {key}", debug=True)
    else:
        _ = agent.log(
            f"Memory delete failed: {deleted_memory_item.error_message}",
            error=True,
            debug=True,
        )
    _ = agent.log(deleted_memory_item, debug=True)

2025-10-13 16:21:31 - circuit_agent - INFO - Memory deleted: thing-to-store
2025-10-13 16:21:31 - circuit_agent - INFO - {
  "success": true,
  "data": {
    "key": "thing-to-store"
  },
  "error": null,
  "error_details": null
}


# Swidge (Swap + Bridge)

In [3]:
# Circuit will try to not filter any quotes, so it is very important to check that the returned quote data is what you would expect and within reason.
# DO NOT just blindly execute all quotes, that could result in very poor execution depending on what markets you are operating in.

execute_fail_test_quote = agent.swidge.quote(
    {
        "from": {
            "network": "ethereum:8453",
            "address": "0xf92b294f79e44f12279db546ed87557c6bd1bc02",
        },
        "to": {
            "network": "ethereum:8453",
            "address": "0xf92b294f79e44f12279db546ed87557c6bd1bc02",
        },
        "fromToken": "0x833589fcd6edb6e08f4c7c32d4f71b54bda02913",
        "toToken": "0x0b3e328455c4059eeb9e3f84b5543f74e24e7e1b",
        "amount": "1000000000000",
        "slippage": "2.0",
    }
)

if not execute_fail_test_quote.success:
    # Handle the error
    _ = agent.log(
        f"Quote failed: {execute_fail_test_quote.error_message}", error=True, debug=True
    )
# else:
#     # Normally execute here, just logging for demo purposes
#     _ = agent.log(execute_fail_test_quote, debug=True)

In [4]:
if abs(float(execute_fail_test_quote.data.priceImpact.percentage)) > 10:
    _ = agent.log(
        f"Warning: Price impact is too high: {execute_fail_test_quote.data.priceImpact.percentage}%",
        error=True,
        debug=True,
    )

2025-10-13 16:52:36 - circuit_agent - ERROR - Warning: Price impact is too high: -10.31%


In [5]:
execute_fail_test = agent.swidge.execute(execute_fail_test_quote.data)
_ = agent.log(execute_fail_test, debug=True)
if not execute_fail_test.success:
    _ = agent.log(
        f"Execute failed: {execute_fail_test.error_message}", error=True, debug=True
    )

2025-10-13 16:52:39 - circuit_agent - INFO - {
  "success": false,
  "data": null,
  "error": "Failed to estimate gas: EstimateGasExecutionError: Execution reverted with reason: ERC20: transfer amount exceeds balance.\n\nEstimate Gas Arguments:\n  from:   0xf92b294f79e44f12279db546ed87557c6bd1bc02\n  to:     0xbbbfd134e9b44bfb5123898ba36b01de7ab93d98\n  value:  0 ETH\n  data:   ...\n\nDetails: execution reverted: ERC20: transfer amount exceeds balance\nVersion: viem@2.35.0",
  "error_details": {
    "error": "Failed to execute quote",
    "message": "Failed to estimate gas: EstimateGasExecutionError: Execution reverted with reason: ERC20: transfer amount exceeds balance.\n\nEstimate Gas Arguments:\n  from:   0xf92b294f79e44f12279db546ed87557c6bd1bc02\n  to:     0xbbbfd134e9b44bfb5123898ba36b01de7ab93d98\n  value:  0 ETH\n  data:   ...\n\nDetails: execution reverted: ERC20: transfer amount exceeds balance\nVersion: viem@2.35.0"
  }
}
2025-10-13 16:52:39 - circuit_agent - ERROR - Execute

In [ ]:
POLYGON_NETWORK = "ethereum:137"
BASE_NETWORK = "ethereum:8453"
USDC_POLYGON_ADDRESS = "0x2791bca1f2de4661ed88a30c99a7a9449aa84174"
WETH_POLYGON_ADDRESS = "0x7ceb23fd6bc0add59e62ac25578270cff1b9f619"

quote = agent.swidge.quote(
    {
        "from": {"network": BASE_NETWORK, "address": agent.sessionWalletAddress},
        "to": {"network": POLYGON_NETWORK, "address": agent.sessionWalletAddress},
        "toToken": USDC_POLYGON_ADDRESS,
        "amount": "100000000000000",
        "slippage": "2.0",
    }
)

if not quote.success:
    _ = agent.log(f"Quote failed: {quote.error_message}", error=True, debug=True)
else:
    execute = agent.swidge.execute(quote.data)
    _ = agent.log(execute, debug=True)

# Polymarket

#### Market Orders

In [ ]:
BRONCOS_JETS_TOKEN_IDS = {
    "BRONCOS": "86192057611122246511563653509192966169513312957180910360241289053249649036697",
    "JETS": "24015698128303510188743058335387340167078113750444014281026927870122555736558",
}

buy_order = agent.platforms.polymarket.market_order(
    {
        "tokenId": BRONCOS_JETS_TOKEN_IDS["BRONCOS"],
        "size": 3,
        "side": "BUY",
    }
)

if buy_order.success:
    _ = agent.log(f"Buy order: {buy_order.data}", debug=True)
else:
    _ = agent.log(
        f"Buy order failed: {buy_order.error_message}", error=True, debug=True
    )

2025-10-11 22:07:55 - circuit_agent - INFO - Buy order: success=True orderInfo=PolymarketOrderInfo(orderId='0x8ee51f310652434f4ed7a5b9bbb2056b660dc1cc88cbee3368428782436a1fe4', side='BUY', size='3.896102', priceUsd='0.77', totalPriceUsd='2.99999854', txHashes=['0x7e8d86487c1ff4423cec74b915f186ab67100a4413b7c4ad47e837e1495e170c'])


In [12]:
sell_order = agent.platforms.polymarket.market_order(
    {
        "tokenId": "43316042420532542168140438864501868551184980388178338105513819372697332591887",  # Steelers vs Bengals - Steelers
        "size": 3,
        "side": "SELL",
    }
)

if sell_order.success:
    _ = agent.log(f"Sell order: {sell_order.data}", debug=True)
else:
    _ = agent.log(
        f"Sell order failed: {sell_order.error_message}", error=True, debug=True
    )

2025-10-13 16:56:23 - circuit_agent - INFO - Sell order: success=True orderInfo=PolymarketOrderInfo(orderId='0x0638521b90a2e8137a55e53d2ee4bee28ce03b889d098b43b988cac4e86868a9', side='SELL', size='3', priceUsd='0.71', totalPriceUsd='2.13', txHashes=['0xa667230980a3269ec2ec898fcd9041429cdc8bca182a71a48596102fb6ef1981'])


#### Redeem Positions

In [14]:
redeem_positions = agent.platforms.polymarket.redeem_positions()


if not redeem_positions.success:
    _ = agent.log(redeem_positions, error=True, debug=True)
else:
    _ = agent.log(redeem_positions, debug=True)

2025-10-13 16:57:08 - circuit_agent - INFO - {
  "success": true,
  "data": null,
  "error": null,
  "error_details": null
}
